<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>

# Assessment: Sentiment Analysis with TAO and Riva
Sentiment analysis is a type of text classification, a common NLP task. 
Using a pretrained language model, such as BERT, it is possible to train a text classification model to classify sentences among defined categories.  In the case of sentiment analysis, there are only two categories: positive and negative.

<img src="images/assess/sentiment_analysis.png">

### Table of Contents
[The Problem](#The-Problem)<br>
[Scoring](#Scoring)<br>
[Step 1: Prepare the Project](#Step-1:-Prepare-the-Project)<br>
[Step 2: Train](#Step-2:-Train)<br>
[Step 3: Infer and Evaluate](#Step-3:-Infer-and-Evaluate)<br>
[Step 4: Export Custom Model](#Step-4:-Export-Custom-Model)<br>
[Step 5: Build and Deploy with Riva](#Step-5:-Build-and-Deploy-with-Riva)<br>
[Step 6: Start Riva Services](#Step-6:-Start-Riva-Services)<br>
[Step 7: Submit You Assessment](#Step-7:-Submit-You-Assessment)<br>

### Notebook Dependencies
The steps in this notebook assume that you have:

1. **NGC Credentials**<br>Be sure you have added your NGC credential as described in the [NGC Setup notebook](003_Intro_NGC_Setup.ipynb).  If you have restarted the course instance, you will need to repeat this step.

In [1]:
# Start fresh...
# Clear Docker containers
!docker kill $(docker ps -q)
# Check for clean environment - this should be empty
!docker ps

91f9425a29df
CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


---
# The Problem

### SST-2 Movie Reviews
The [Stanford Sentiment Treebank v2 (SST-2)](https://nlp.stanford.edu/sentiment/index.html) dataset is a corpus with fully labeled (two classes: positive and negative) single sentences extracted from movie reviews. Your task is to train a model using the dataset and deploy it to Riva, where you can run inference using the Riva API.

### Your Project
You are provided with labeled training and validation datasets, `train_small.tsv` and `dev_small.tsv` for the project.  There is also a test set, `test.tsv`, for a final test of the model.  All datasets are contained in the `tao/data/SST-2` directory.  You can open any of these files to take a look at the actual data and format:
* [train_small.tsv](tao/data/SST-2/train_small.tsv)
* [dev_small.tsv](tao/data/SST-2/dev_small.tsv)
* [test.tsv](tao/data/SST-2/test.tsv)

Your assignment is to train a [text classification model](https://docs.nvidia.com/tao/tao-toolkit/text/nlp/text_classification.html) with TAO using the `tao text_classification` launch command. After training, you must export the custom model, then deploy it using Riva.  

---
# Scoring
You will be assessed on your ability to effectively and efficiently train and deploy the model.  This coding assessment is worth 70 points, divided as follows:

### Rubric

| Step                    | Graded                                                 | FIXMEs?  | Points |
|-------------------------|--------------------------------------------------------|----------|--------|
| 1. Prepare the Project  | Specs and path definitions (spec files are present)    |  1       | 5      |
| 2. Train                | Efficient training parameters (faster training)        |  5       | 15     |
| 3. Infer and Evaluate   | Achieve good inference performance (F1 value >= 88)    |  0       | 10     |
| 4. Export Custom Model  | Export for Riva (model exported in correct format)   |  1       | 12     |
| 5. Build and Deploy     | Riva ServiceMaker (correct models built and loaded)  |  2       | 14     |
| 6. Start Riva         | Riva Server (correct config; models run)             |  1       | 14     |

Although you are very capable at this point of building the project without any help at all, some scaffolding is provided, including specific names for variables.  This is for the benefit of the autograder, so please use these constructs for your assessment.  In addition, a copy of the latest output for your executed cells in some cases is saved in the `my_assessment` directory.  Along the way, there are a few opportunities to check your work to see if you are on the right track. 

Once you are confident that you've built a reliable model, follow the instructions for submission at the end of the notebook.

### Resources and Hints

* **[TAO User's Guide](https://docs.nvidia.com/tao/tao-toolkit/index.html)**<br>
* **[Riva Speech Skills User's Guide](https://docs.nvidia.com/deeplearning/riva/user-guide/docs/index.html)**<br>
* **TAO Example**<br>
Review what you've learned in the [NER Fine-Tuning](007_NLP_Finetune_NER.ipynb) notebook to train, infer, evaluate, and export a TAO model.  The `tao token_classification` commands are very similar to the `tao text_classification` commands.
* **Riva Deployment Example**<br>
Review what you've learned in the [NER Model Deployment with Riva](008_NLP_Deploy_NER.ipynb) notebook to build and deploy the model, as well as start the Riva server.
* **AMP Optimization Level (trainer.amp_level):**<br>
To use mixed precision, set AMP to 'O1' or 'O2'; to train without mixed precision, set it to 'O0'.
* **Precision (trainer.precision):**<br>
To speed up training, you can set the precision to 16 instead of the standard 32 with little or no loss in accuracy.
* **Number of epochs (trainer.max_epochs):**<br>
The project is designed so that you should achieve success on this dataset with only 2 epochs, but feel free to run more. On a Tesla T4, this takes 5-6 minutes if you run it efficiently!

---
# Step 1: Prepare the Project

### Set up Project Paths (not graded)
This block is complete, but feel free to add to it.

In [2]:
# Set the TAO paths for the project
##### TAO paths - source
SOURCE_MOUNT="/dli/task/tao"
DESTINATION_MOUNT = "/workspace/mount"

##### TAO paths - source
# Define location of the SST-2 dataset
DATA = SOURCE_MOUNT+'/data/SST-2'
# Directory where the .riva model is stored
EXPORT_MODEL_LOC = SOURCE_MOUNT + '/results/sst2/export'

##### TAO paths - destination (from the perspective of the TAO Docker)
# The path to the specification YAML 
SPECS_DIR = DESTINATION_MOUNT + '/specs'
# The results are saved at this path by default
RESULTS_DIR = DESTINATION_MOUNT + '/results'
# The data are saved at this path by default
DATA_DIR = DESTINATION_MOUNT + '/data'
# The results are saved at this path by default
MODELS_DIR = DESTINATION_MOUNT + '/models'

# Set your encryption key, and use the same key for all commands. Please use "tlt_encode" if you'd like to deploy the models later with NVIDIA Riva.
KEY = 'tlt_encode'

### Get the Spec Files (graded)
Complete the <i><strong style="color:green;">#FIXME</strong></i> line(s) and run the cell.

In [4]:
import os
from shutil import rmtree

# Delete the specs directory if it already exists
folder = SOURCE_MOUNT + '/specs'
if os.path.exists(folder):
    rmtree(folder)

# Get the text classification task spec files
!tao text_classification download_specs\
    -o $SPECS_DIR/text_classification \
    -r $RESULTS_DIR \
    2>&1|tee my_assessment/step1.txt # DO NOT REMOVE THIS LINE

2022-03-05 05:22:19,854 [INFO] root: Registry: ['nvcr.io']
2022-03-05 05:22:19,964 [WARNING] tlt.components.docker_handler.docker_handler: 
Docker will run the commands as root. If you would like to retain your
local host permissions, please add the "user":"UID:GID" in the
DockerOptions portion of the "/root/.tao_mounts.json" file. You can obtain your
users UID and GID by using the "id -u" and "id -g" commands on the
terminal.
[NeMo W 2022-03-05 05:22:23 experimental:27] Module <class 'nemo.collections.nlp.modules.common.megatron.megatron_bert.MegatronBertEncoder'> is experimental, not ready for production and is not fully supported. Use at your own risk.
INFO: Generating new fontManager, this may take some time...
[NeMo I 2022-03-05 05:22:25 tlt_logging:20] Experiment configuration:
    exp_manager:
      task_name: download_specs
      explicit_log_dir: /workspace/mount/results
    source_data_dir: /opt/conda/lib/python3.8/site-packages/nlp/text_classification/experiment_specs
    ta

---
# Step 2: Train
### Run the Trainer (graded)
Review the `train.yaml` file you've just downloaded. Run the trainer in TAO and override YAML config values as necessary.

Complete the <i><strong style="color:green;">#FIXME</strong></i> line(s) and run the cell. Feel free to add/remove override values as you see fit.

In [7]:
%%time
# For BERT training on SST-2:
!tao text_classification train \
    -e $SPECS_DIR/text_classification/train.yaml \
    -g 1  \
    -k $KEY \
    -r $RESULTS_DIR/sst2 \
    training_ds.file_path=$DATA_DIR/SST-2/train.tsv \
    validation_ds.file_path=$DATA_DIR/SST-2/dev.tsv \
    model.class_labels.class_labels_file=$DATA_DIR/SST-2/label_ids.csv \
    trainer.amp_level='O1' \
    trainer.precision=16 \
    trainer.max_epochs=2 \
    2>&1|tee my_assessment/step2.txt # DO NOT REMOVE THIS LINE

2022-03-05 05:43:15,881 [INFO] root: Registry: ['nvcr.io']
2022-03-05 05:43:15,996 [WARNING] tlt.components.docker_handler.docker_handler: 
Docker will run the commands as root. If you would like to retain your
local host permissions, please add the "user":"UID:GID" in the
DockerOptions portion of the "/root/.tao_mounts.json" file. You can obtain your
users UID and GID by using the "id -u" and "id -g" commands on the
terminal.
[NeMo W 2022-03-05 05:43:19 experimental:27] Module <class 'nemo.collections.nlp.modules.common.megatron.megatron_bert.MegatronBertEncoder'> is experimental, not ready for production and is not fully supported. Use at your own risk.
INFO: Generating new fontManager, this may take some time...
[NeMo W 2022-03-05 05:43:22 experimental:27] Module <class 'nemo.collections.nlp.modules.common.megatron.megatron_bert.MegatronBertEncoder'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo I 2022-03-05 05:43:23 tlt_logging:20

The train command produces a model file called `trained-model.tlt` saved at `results/sst2/checkpoints/trained-model.tlt`. 

---
# Step 3: Infer and Evaluate

### Create the Queries (not graded)
Execute the following cell to create queries for inference.

In [8]:
%%writefile $SOURCE_MOUNT/specs/text_classification/infer.yaml

# Copyright (c) 2020, NVIDIA CORPORATION. All rights reserved.
# TAO Spec file for inference using a previously pretrained BERT model for a text classification task.

# "Simulate" user input: batch with four samples.
input_batch:
- "this is a good script , good dialogue , funny even for adults ."
- "the affectionate loopiness that once seemed congenital to demme s perspective has a tough time emerging from between the badly dated cutesy-pie mystery scenario a nd the newfangled hollywood post-production effects ."
- " this piece of channel 5 grade trash is , quite frankly , an insult to the intelligence of the true genre enthusiast . "
- "a delightful coming-of-age story ."

Overwriting /dli/task/tao/specs/text_classification/infer.yaml


### Run Inference on the Trained Model (not graded)

In [9]:
# Run inference on user data:
!tao text_classification infer \
    -e $SPECS_DIR/text_classification/infer.yaml \
    -g 1 \
    -m $RESULTS_DIR/sst2/checkpoints/trained-model.tlt \
    -k $KEY \
    -r $RESULTS_DIR/sst2/infer

2022-03-05 05:57:53,423 [INFO] root: Registry: ['nvcr.io']
2022-03-05 05:57:53,577 [WARNING] tlt.components.docker_handler.docker_handler: 
Docker will run the commands as root. If you would like to retain your
local host permissions, please add the "user":"UID:GID" in the
DockerOptions portion of the "/root/.tao_mounts.json" file. You can obtain your
users UID and GID by using the "id -u" and "id -g" commands on the
terminal.
[NeMo W 2022-03-05 05:57:57 experimental:27] Module <class 'nemo.collections.nlp.modules.common.megatron.megatron_bert.MegatronBertEncoder'> is experimental, not ready for production and is not fully supported. Use at your own risk.
INFO: Generating new fontManager, this may take some time...
[NeMo W 2022-03-05 05:58:01 experimental:27] Module <class 'nemo.collections.nlp.modules.common.megatron.megatron_bert.MegatronBertEncoder'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo I 2022-03-05 05:58:01 tlt_logging:20

### Evaluate your Model (results graded)
Execute the following cell without changes.  Review your output to see if you had an F1 result above the 88% goal.  If not, you may need to retrain your model.

In [10]:
# For BERT evaluation on SST-2:
!tao text_classification evaluate \
    -e $SPECS_DIR/text_classification/evaluate.yaml \
    -g 1 \
    -m $RESULTS_DIR/sst2/checkpoints/trained-model.tlt \
    -k $KEY \
    -r $RESULTS_DIR/sst2/eval \
    test_ds.file_path=$DATA_DIR/SST-2/test.tsv \
    test_ds.batch_size=32 \
    test_ds.num_samples=-1 \
    2>&1|tee my_assessment/step3.txt # DO NOT REMOVE THIS LINE

2022-03-05 05:58:32,986 [INFO] root: Registry: ['nvcr.io']
2022-03-05 05:58:33,082 [WARNING] tlt.components.docker_handler.docker_handler: 
Docker will run the commands as root. If you would like to retain your
local host permissions, please add the "user":"UID:GID" in the
DockerOptions portion of the "/root/.tao_mounts.json" file. You can obtain your
users UID and GID by using the "id -u" and "id -g" commands on the
terminal.
[NeMo W 2022-03-05 05:58:36 experimental:27] Module <class 'nemo.collections.nlp.modules.common.megatron.megatron_bert.MegatronBertEncoder'> is experimental, not ready for production and is not fully supported. Use at your own risk.
INFO: Generating new fontManager, this may take some time...
[NeMo W 2022-03-05 05:58:40 experimental:27] Module <class 'nemo.collections.nlp.modules.common.megatron.megatron_bert.MegatronBertEncoder'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo I 2022-03-05 05:58:40 tlt_logging:20

---
# Step 4: Export Custom Model
### Export the Model for Riva (graded)
Complete the <i><strong style="color:green;">#FIXME</strong></i> line(s) and run the cell.

In [11]:
#  For export to Riva:
!tao text_classification export \
    -e $SPECS_DIR/text_classification/export.yaml \
    -g 1 \
    -m $RESULTS_DIR/sst2/checkpoints/trained-model.tlt \
    -k $KEY \
    -r $RESULTS_DIR/sst2/export/ \
    export_format=RIVA \
    export_to=tc-model.riva \
    2>&1|tee my_assessment/step4.txt # DO NOT REMOVE THIS LINE

2022-03-05 05:59:59,759 [INFO] root: Registry: ['nvcr.io']
2022-03-05 05:59:59,872 [WARNING] tlt.components.docker_handler.docker_handler: 
Docker will run the commands as root. If you would like to retain your
local host permissions, please add the "user":"UID:GID" in the
DockerOptions portion of the "/root/.tao_mounts.json" file. You can obtain your
users UID and GID by using the "id -u" and "id -g" commands on the
terminal.
[NeMo W 2022-03-05 06:00:03 experimental:27] Module <class 'nemo.collections.nlp.modules.common.megatron.megatron_bert.MegatronBertEncoder'> is experimental, not ready for production and is not fully supported. Use at your own risk.
INFO: Generating new fontManager, this may take some time...
[NeMo W 2022-03-05 06:00:06 experimental:27] Module <class 'nemo.collections.nlp.modules.common.megatron.megatron_bert.MegatronBertEncoder'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo I 2022-03-05 06:00:07 tlt_logging:20

In [12]:
# Check your work - does the exported tc-model.riva model exist?
!ls $EXPORT_MODEL_LOC

export.log  tc-model.riva


---
# Step 5: Build and Deploy with Riva
### Set up Project Paths (not graded)
This block is complete, but feel free to add to it.

In [13]:
# Set the Riva paths for the project
WORKSPACE = "/dli/task"

##### Riva Paths
# ServiceMaker Docker
RIVA_SM_CONTAINER = "nvcr.io/nvidia/riva/riva-speech:1.4.0-beta-servicemaker"

# Model output directories
RMIR_LOC = WORKSPACE + "/riva/riva_quickstart/models_repo_assessment/rmir"
RIVA_MODEL_LOC = WORKSPACE + '/riva/riva_quickstart/models_repo_assessment'

# Model Names
EXPORT_MODEL_NAME = "tc-model.riva"  
RMIR_MODEL_NAME = "tc-model.rmir"

# Riva Quick Start 
RIVA_QS = WORKSPACE + "/riva/riva_quickstart"

### Build and Deploy with Riva ServiceMaker (graded)
Complete the <i><strong style="color:green;">#FIXME</strong></i> line(s) and run the cells.

In [14]:
# Syntax: riva-build <task-name> output-dir-for-rmir/model.rmir:key dir-for-riva/model.riva:key
!docker run --rm --gpus 1 \
    -v $EXPORT_MODEL_LOC:/tao \
    -v $RMIR_LOC:/riva \
    $RIVA_SM_CONTAINER -- \
    riva-build text_classification \
    -f /riva/$RMIR_MODEL_NAME:$KEY /tao/$EXPORT_MODEL_NAME:$KEY \
    2>&1|tee my_assessment/step5.txt # DO NOT REMOVE THIS LINE


=== Riva Speech Skills ===

NVIDIA Release devel (build 22382700)

Copyright (c) 2018-2021, NVIDIA CORPORATION.  All rights reserved.

Various files include modifications (c) NVIDIA CORPORATION.  All rights reserved.
NVIDIA modifications are covered by the license terms that apply to the underlying
project or file.

NOTE: Legacy NVIDIA Driver detected.  Compatibility mode ENABLED.

NOTE: The SHMEM allocation limit is set to the default of 64MB.  This may be
   insufficient for the inference server.  NVIDIA recommends the use of the following flags:
   nvidia-docker run --shm-size=1g --ulimit memlock=-1 --ulimit stack=67108864 ...

2022-03-05 06:03:47,638 [ERROR] Condition for key 'runtime' (PyTorch  <built-in function eq> ONNX) is not fulfilled
2022-03-05 06:03:47,705 [INFO] Packing binaries for self
2022-03-05 06:03:48,886 [ERROR] Condition for key 'runtime' (PyTorch  <built-in function eq> ONNX) is not fulfilled
2022-03-05 06:03:48,949 [INFO] Trying to extract from model tc-model.ri

In [15]:
# Check your work - does the exported tc-model.rmir model exist?
!ls $RMIR_LOC

tc-model.rmir


In [16]:
# Syntax: riva-deploy -f dir-for-rmir/model.rmir:key output-dir-for-repository
!docker run --rm --gpus 1 \
    -v $RIVA_MODEL_LOC:/data \
    $RIVA_SM_CONTAINER -- \
    riva-deploy -f  /data/rmir/$RMIR_MODEL_NAME:$KEY \
    /data/models/ \
    2>&1|tee -a my_assessment/step5.txt # DO NOT REMOVE THIS LINE


=== Riva Speech Skills ===

NVIDIA Release devel (build 22382700)

Copyright (c) 2018-2021, NVIDIA CORPORATION.  All rights reserved.

Various files include modifications (c) NVIDIA CORPORATION.  All rights reserved.
NVIDIA modifications are covered by the license terms that apply to the underlying
project or file.

NOTE: Legacy NVIDIA Driver detected.  Compatibility mode ENABLED.

NOTE: The SHMEM allocation limit is set to the default of 64MB.  This may be
   insufficient for the inference server.  NVIDIA recommends the use of the following flags:
   nvidia-docker run --shm-size=1g --ulimit memlock=-1 --ulimit stack=67108864 ...

2022-03-05 06:07:07,471 [INFO] Writing Riva model repository to '/data/models/'...
2022-03-05 06:07:07,471 [INFO] The riva model repo target directory is /data/models/
2022-03-05 06:07:08,665 [INFO] Extract_binaries for tokenizer -> /data/models/riva_tokenizer/1
2022-03-05 06:07:08,667 [INFO] Extract_binaries for language_model -> /data/models/riva-trt-riva_

In [17]:
# Check your work - are there optimized models for text classification?
!ls $RIVA_MODEL_LOC/models

riva-trt-riva_text_classification_default-nn-bert-base-uncased	riva_tokenizer
riva_text_classification_default


---
# Step 6: Start Riva Services

### Configure and Start Riva (graded)
Next, modify the [config.sh](riva/riva_quickstart/config.sh) to enable relevant Riva services. 
In this case, we want to start NLP services, provide the encryption key, and update the path to the model repository (`RIVA_MODEL_LOC`). 
Open the [config.sh](riva/riva_quickstart/config.sh) and make changes where necessary, then start the server.

In [19]:
# Run Riva Start. This will deploy the model.
!cd $RIVA_QS && bash riva_start.sh config.sh

Starting Riva Speech Services. This may take several minutes depending on the number of models deployed.
Waiting for Riva server to load all models...retrying in 10 seconds
Waiting for Riva server to load all models...retrying in 10 seconds
Riva server is ready...


In [20]:
# Check Riva running services 
!docker logs riva-speech \
    2>&1|tee my_assessment/step6.txt # DO NOT REMOVE THIS LINE


=== Riva Speech Skills ===

NVIDIA Release 21.07 (build 25292380)

Copyright (c) 2018-2021, NVIDIA CORPORATION.  All rights reserved.

Various files include modifications (c) NVIDIA CORPORATION.  All rights reserved.
NVIDIA modifications are covered by the license terms that apply to the underlying
project or file.

NOTE: Legacy NVIDIA Driver detected.  Compatibility mode ENABLED.

NOTE: The SHMEM allocation limit is set to the default of 64MB.  This may be
   insufficient for the inference server.  NVIDIA recommends the use of the following flags:
   nvidia-docker run --shm-size=1g --ulimit memlock=-1 --ulimit stack=67108864 ...

  > Riva waiting for Triton server to load all models...retrying in 1 second
I0305 06:10:10.043343 73 metrics.cc:228] Collecting metrics for GPU 0: Tesla T4
I0305 06:10:10.092016 73 onnxruntime.cc:1722] TRITONBACKEND_Initialize: onnxruntime
I0305 06:10:10.093050 73 onnxruntime.cc:1732] Triton TRITONBACKEND API version: 1.0
I0305 06:10:10.093065 73 onnxruntim

### Riva Service Request (not graded)
Although the SST-2 data set is trained on movie sentiments, it will likely work in our restaurant domain too.  Give it a try with the following queries or make up your own!

In [ ]:
%run my_assessment/sentiment_analysis_client.py --query "I like pizza"
%run my_assessment/sentiment_analysis_client.py --query "I don't like this restaurant"
%run my_assessment/sentiment_analysis_client.py --query "yeah, sounds good"

### Stop Riva Services 

In [ ]:
# Shut down Riva 
!bash $RIVA_QS/riva_stop.sh

---
# Step 7: Submit You Assessment
How were your results? 

If you are satisfied that you have completed the code correctly, and that your training and deployment are correct, you can submit your project as follows to the autograder:

1. Go back to the GPU launch page and click the checkmark to run the assessment:

<img src="images/assess/assessment_checkmark.png" width=800>

2. That's it!  If you passed, you'll receive a pop-up window saying so, and the points will be credited to your progress.  If not, you'll receive feedback in the pop-up window. 

<img src="images/assess/assessment_pass_popup.png" width=800>

You can always check your assessment progress in the course progress tab.  Note that partial values for the coding assessment won't be visible here - it shows up as either 0 or 70 points.  Be sure to complete the questions on Transformer and Deployment on the same course page to qualify for your final certificate!

<img src="images/assess/progress.png" width=800>

<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>